In [2]:
import tensorflow as tf

import sys
sys.path.append("..")

from models.classification_model import ClassificationDGP
from likelihoods import Softmax
from utils_training import classification_train
# setting dataset
record_file = 'MNIST_results.txt'
repeat = 3 # run the task several times
print('#'*40, f"Results stored in {record_file}.", '#'*40)

dataset_name_list = ['mnist']
    # , 'wine_red', 'wine_white', 'concrete',
    #                  'energy', 'kin8nm', 'naval', 'power','protein']
d_in_list = [28*28]
    # , 11, 11, 8, 8, 8, 16, 4, 9]
d_out = 10
# full Bayesian training settings
batch_size = 200
lr_0 = 0.1
beta = 0.0
total_epochs = 2000
start_sampling_epoch = 150
epochs_per_cycle = 50
print_epoch_cycle = 50
drop_mixing = 17
for i in range(len(dataset_name_list)):
    dataset_name = dataset_name_list[i]
    hid_n_gp = 30

    log_p_coll = []
    acc_coll = []
    for run_index in range(repeat):
        print('*' * 70)
        print('*' * 30, '2-layer DGPs ', f'Run {run_index}', '*' * 30)
        model = ClassificationDGP(d_in_list[i], d_out, n_hidden_layers=2, n_rf=100,
                                  n_gp=[hid_n_gp, d_out], likelihood=Softmax(),
                                  kernel_type_list=['RBF', 'RBF'], kernel_trainable=True,
                                  random_fixed=True, set_nonzero_mean=False, input_cat=True)
        # return matrix [S, N]
        log_p, acc = classification_train(model, dataset_name=dataset_name, batch_size=batch_size,
                                          lr_0=lr_0, momentum_decay=beta, full_bayesian=True,
                                          precond_type='identity',
                                          resample_in_cycle_head=False,
                                          total_epochs=total_epochs, start_sampling_epoch=start_sampling_epoch,
                                          epochs_per_cycle=epochs_per_cycle,
                                          print_epoch_cycle=print_epoch_cycle)
        log_p_droped = log_p[drop_mixing:, :]
        acc_droped = acc[drop_mixing:]

        n_models_droped = tf.shape(acc_droped)[0]
        predict_log_p = tf.reduce_logsumexp(log_p_droped, axis=0) - tf.math.log(tf.cast(n_models_droped, tf.float32))
        predict_log_p = tf.reduce_mean(predict_log_p)
        predict_acc = tf.reduce_mean(acc_droped)

        print(f"Dataset: {dataset_name}, Full Bayes ")
        print(f"Number of sampled models(after dropping {drop_mixing} samples): {n_models_droped}")
        print(f"Test Log Likelihood of all sampled models: {predict_log_p}")
        print(f"Test Root MSE of all sampled models: {predict_acc}")

        log_p_coll.append(predict_log_p)
        acc_coll.append(predict_acc)
        print('*' * 70)
        print('*' * 70, "\n")

    log_p_coll = tf.concat(log_p_coll, axis=0)
    acc_coll = tf.concat(acc_coll, axis=0)
    with open(record_file, 'a') as f:
        f.write(f"Dataset: {dataset_name}, with no preconditioner\n")
        f.write(f"Predict Mean Log Likelihood: {log_p_coll},\n  ")
        f.write(f"Their mean is: {tf.reduce_mean(log_p_coll)}, ")
        f.write(f"std is: {tf.math.reduce_std(log_p_coll)}\n")
        f.write(f"Predict Mean Acc: {acc_coll},\n  ")
        f.write(f"Their mean is: {tf.reduce_mean(acc_coll)}, ")
        f.write(f"std is: {tf.math.reduce_std(acc_coll)}\n\n")
# with rmsprop preconditioner
# full Bayesian training settings
batch_size = 200
lr_0 = 0.1
beta = 0.0
total_epochs = 2000
start_sampling_epoch = 150
epochs_per_cycle = 50
print_epoch_cycle = 25
drop_mixing = 17
for i in range(len(dataset_name_list)):
    dataset_name = dataset_name_list[i]
    hid_n_gp = 30

    log_p_coll = []
    acc_coll = []
    for run_index in range(repeat):
        print('*' * 70)
        print('*' * 30, '2-layer DGPs ', f'Run {run_index}', '*' * 30)
        model = ClassificationDGP(d_in_list[i], d_out, n_hidden_layers=2, n_rf=100,
                                  n_gp=[hid_n_gp, d_out], likelihood=Softmax(),
                                  kernel_type_list=['RBF', 'RBF'], kernel_trainable=True,
                                  random_fixed=True, set_nonzero_mean=False, input_cat=True)
        # return matrix [S, N]
        log_p, acc = classification_train(model, dataset_name=dataset_name, batch_size=batch_size,
                                          lr_0=lr_0, momentum_decay=beta, full_bayesian=True,
                                          precond_type='rmsprop', K_batches=32,second_moment_centered=False,
                                          resample_in_cycle_head=False,
                                          total_epochs=total_epochs, start_sampling_epoch=start_sampling_epoch,
                                          epochs_per_cycle=epochs_per_cycle,
                                          print_epoch_cycle=print_epoch_cycle)
        log_p_droped = log_p[drop_mixing:, :]
        acc_droped = acc[drop_mixing:]

        n_models_droped = tf.shape(acc_droped)[0]
        predict_log_p = tf.reduce_logsumexp(log_p_droped, axis=0) - tf.math.log(tf.cast(n_models_droped, tf.float32))
        predict_log_p = tf.reduce_mean(predict_log_p)
        predict_acc = tf.reduce_mean(acc_droped)

        print(f"Dataset: {dataset_name}, Full Bayes ")
        print(f"Number of sampled models(after dropping {drop_mixing} samples): {n_models_droped}")
        print(f"Test Log Likelihood of all sampled models: {predict_log_p}")
        print(f"Test Root MSE of all sampled models: {predict_acc}")

        log_p_coll.append(predict_log_p)
        acc_coll.append(predict_acc)
        print('*' * 70)
        print('*' * 70, "\n")

    log_p_coll = tf.concat(log_p_coll, axis=0)
    acc_coll = tf.concat(acc_coll, axis=0)
    with open(record_file, 'a') as f:
        f.write(f"Dataset: {dataset_name}, with RMSProp preconditioner \n")
        f.write(f"Predict Mean Log Likelihood: {log_p_coll},\n  ")
        f.write(f"Their mean is: {tf.reduce_mean(log_p_coll)}, ")
        f.write(f"std is: {tf.math.reduce_std(log_p_coll)}\n")
        f.write(f"Predict Mean Acc: {acc_coll},\n  ")
        f.write(f"Their mean is: {tf.reduce_mean(acc_coll)}, ")
        f.write(f"std is: {tf.math.reduce_std(acc_coll)}\n\n")
# fashion mnist
# setting dataset
record_file = 'Fashion_MNIST_results.txt'
repeat = 1 # run the task several times
print('#'*40, f"Results stored in {record_file}.", '#'*40)

dataset_name_list = ['fashion_mnist']
    # , 'wine_red', 'wine_white', 'concrete',
    #                  'energy', 'kin8nm', 'naval', 'power','protein']
d_in_list = [28*28]
    # , 11, 11, 8, 8, 8, 16, 4, 9]
d_out = 10
# full Bayesian training settings
batch_size = 200
lr_0 = 0.1
beta = 0.0
total_epochs = 2000
start_sampling_epoch = 150
epochs_per_cycle = 50
print_epoch_cycle = 25
drop_mixing = 17
for i in range(len(dataset_name_list)):
    dataset_name = dataset_name_list[i]
    hid_n_gp = 30

    log_p_coll = []
    acc_coll = []
    for run_index in range(repeat):
        print('*' * 70)
        print('*' * 30, '2-layer DGPs ', f'Run {run_index}', '*' * 30)
        model = ClassificationDGP(d_in_list[i], d_out, n_hidden_layers=2, n_rf=100,
                                  n_gp=[hid_n_gp, d_out], likelihood=Softmax(),
                                  kernel_type_list=['RBF', 'RBF'], kernel_trainable=True,
                                  random_fixed=True, set_nonzero_mean=False, input_cat=True)
        # return matrix [S, N]
        log_p, acc = classification_train(model, dataset_name=dataset_name, batch_size=batch_size,
                                          lr_0=lr_0, momentum_decay=beta, full_bayesian=True,
                                          precond_type='identity',
                                          resample_in_cycle_head=False,
                                          total_epochs=total_epochs, start_sampling_epoch=start_sampling_epoch,
                                          epochs_per_cycle=epochs_per_cycle,
                                          print_epoch_cycle=print_epoch_cycle)
        log_p_droped = log_p[drop_mixing:, :]
        acc_droped = acc[drop_mixing:]

        n_models_droped = tf.shape(acc_droped)[0]
        predict_log_p = tf.reduce_logsumexp(log_p_droped, axis=0) - tf.math.log(tf.cast(n_models_droped, tf.float32))
        predict_log_p = tf.reduce_mean(predict_log_p)
        predict_acc = tf.reduce_mean(acc_droped)

        print(f"Dataset: {dataset_name}, Full Bayes ")
        print(f"Number of sampled models(after dropping {drop_mixing} samples): {n_models_droped}")
        print(f"Test Log Likelihood of all sampled models: {predict_log_p}")
        print(f"Test Root MSE of all sampled models: {predict_acc}")

        log_p_coll.append(predict_log_p)
        acc_coll.append(predict_acc)
        print('*' * 70)
        print('*' * 70, "\n")

    log_p_coll = tf.concat(log_p_coll, axis=0)
    acc_coll = tf.concat(acc_coll, axis=0)
    with open(record_file, 'a') as f:
        f.write(f"Dataset: {dataset_name}, with no preconditioner \n")
        f.write(f"Predict Mean Log Likelihood: {log_p_coll},\n  ")
        f.write(f"Their mean is: {tf.reduce_mean(log_p_coll)}, ")
        f.write(f"std is: {tf.math.reduce_std(log_p_coll)}\n")
        f.write(f"Predict Mean Acc: {acc_coll},\n  ")
        f.write(f"Their mean is: {tf.reduce_mean(acc_coll)}, ")
        f.write(f"std is: {tf.math.reduce_std(acc_coll)}\n\n")
# with RMSProp preconditioner
# setting dataset
record_file = 'Fashion_MNIST_results.txt'
repeat = 2 # run the task several times
print('#'*40, f"Results stored in {record_file}.", '#'*40)

dataset_name_list = ['fashion_mnist']
    # , 'wine_red', 'wine_white', 'concrete',
    #                  'energy', 'kin8nm', 'naval', 'power','protein']
d_in_list = [28*28]
    # , 11, 11, 8, 8, 8, 16, 4, 9]
d_out = 10
# full Bayesian training settings
batch_size = 200
lr_0 = 0.1
beta = 0.0
total_epochs = 2000
start_sampling_epoch = 150
epochs_per_cycle = 50
print_epoch_cycle = 25
drop_mixing = 17
for i in range(len(dataset_name_list)):
    dataset_name = dataset_name_list[i]
    hid_n_gp = 30

    log_p_coll = []
    acc_coll = []
    for run_index in range(repeat):
        print('*' * 70)
        print('*' * 30, '2-layer DGPs ', f'Run {run_index}', '*' * 30)
        model = ClassificationDGP(d_in_list[i], d_out, n_hidden_layers=2, n_rf=100,
                                  n_gp=[hid_n_gp, d_out], likelihood=Softmax(),
                                  kernel_type_list=['RBF', 'RBF'], kernel_trainable=True,
                                  random_fixed=True, set_nonzero_mean=False, input_cat=True)
        # return matrix [S, N]
        log_p, acc = classification_train(model, dataset_name=dataset_name, batch_size=batch_size,
                                          lr_0=lr_0, momentum_decay=beta, full_bayesian=True,
                                          precond_type='rmsprop', K_batches=32,second_moment_centered=False,
                                          resample_in_cycle_head=False,
                                          total_epochs=total_epochs, start_sampling_epoch=start_sampling_epoch,
                                          epochs_per_cycle=epochs_per_cycle,
                                          print_epoch_cycle=print_epoch_cycle)
        log_p_droped = log_p[drop_mixing:, :]
        acc_droped = acc[drop_mixing:]

        n_models_droped = tf.shape(acc_droped)[0]
        predict_log_p = tf.reduce_logsumexp(log_p_droped, axis=0) - tf.math.log(tf.cast(n_models_droped, tf.float32))
        predict_log_p = tf.reduce_mean(predict_log_p)
        predict_acc = tf.reduce_mean(acc_droped)

        print(f"Dataset: {dataset_name}, Full Bayes ")
        print(f"Number of sampled models(after dropping {drop_mixing} samples): {n_models_droped}")
        print(f"Test Log Likelihood of all sampled models: {predict_log_p}")
        print(f"Test Root MSE of all sampled models: {predict_acc}")

        log_p_coll.append(predict_log_p)
        acc_coll.append(predict_acc)
        print('*' * 70)
        print('*' * 70, "\n")

    log_p_coll = tf.concat(log_p_coll, axis=0)
    acc_coll = tf.concat(acc_coll, axis=0)
    with open(record_file, 'a') as f:
        f.write(f"Dataset: {dataset_name}, with RMSProp preconditioner \n")
        f.write(f"Predict Mean Log Likelihood: {log_p_coll},\n  ")
        f.write(f"Their mean is: {tf.reduce_mean(log_p_coll)}, ")
        f.write(f"std is: {tf.math.reduce_std(log_p_coll)}\n")
        f.write(f"Predict Mean Acc: {acc_coll},\n  ")
        f.write(f"Their mean is: {tf.reduce_mean(acc_coll)}, ")
        f.write(f"std is: {tf.math.reduce_std(acc_coll)}\n\n")
